In [67]:
import pandas as pd
import numpy as np
from pathlib import Path


#  Load the raw dataset

RAW_PATH = Path(r"C:\Users\pooja\OneDrive\Desktop\PoojaPattu_E_DA_asses\UberDataset.csv")
CLEAN_PATH = Path(r"C:\Users\pooja\OneDrive\Desktop\PoojaPattu_E_DA_asses\UberDataset_Cleaned.csv")

print("🔹 Loading dataset...")
df = pd.read_csv(RAW_PATH)

print(f"Initial shape: {df.shape}")
print("Columns:", list(df.columns))
print("=" * 60)

🔹 Loading dataset...
Initial shape: (420, 11)
Columns: ['START_DATE', 'END_DATE', 'CATEGORY', 'START', 'STOP', 'MILES', 'PURPOSE', 'TRIP_DURATION_MIN', 'MONTH', 'DAY_OF_WEEK', 'HOUR']


In [69]:
# Inspect for data issues


# Summary statistics
print("Dataset Info:\n")
df.info()
print("\nMissing values per column:\n", df.isna().sum())
print("\nDuplicate rows:", df.duplicated().sum())


Dataset Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   START_DATE         420 non-null    object 
 1   END_DATE           420 non-null    object 
 2   CATEGORY           420 non-null    object 
 3   START              420 non-null    object 
 4   STOP               420 non-null    object 
 5   MILES              420 non-null    float64
 6   PURPOSE            420 non-null    object 
 7   TRIP_DURATION_MIN  420 non-null    float64
 8   MONTH              420 non-null    object 
 9   DAY_OF_WEEK        420 non-null    object 
 10  HOUR               420 non-null    int64  
dtypes: float64(2), int64(1), object(8)
memory usage: 36.2+ KB

Missing values per column:
 START_DATE           0
END_DATE             0
CATEGORY             0
START                0
STOP                 0
MILES                0
PURPOSE              0
TRI

In [71]:
# Handle missing data

# 'PURPOSE' has many missing values (~40%+), but we retain them for EDA purposes.
# We'll fill missing purposes with "Unknown" to avoid dropping informative rows.

df['PURPOSE'] = df['PURPOSE'].fillna("Unknown")

# For CATEGORY, START, STOP, END_DATE – very few nulls, so we drop those rows safely.
df = df.dropna(subset=['END_DATE', 'CATEGORY', 'START', 'STOP'])

print(f"After handling nulls: {df.shape}")


After handling nulls: (420, 11)


In [73]:
# Handle duplicates

before_dupes = df.shape[0]
df = df.drop_duplicates()
after_dupes = df.shape[0]
print(f"Removed {before_dupes - after_dupes} duplicate rows.")

Removed 0 duplicate rows.


In [75]:
# Convert data types

# Convert date strings to datetime objects
df['START_DATE'] = pd.to_datetime(df['START_DATE'], errors='coerce')
df['END_DATE'] = pd.to_datetime(df['END_DATE'], errors='coerce')

# Ensure numeric type for miles
df['MILES'] = pd.to_numeric(df['MILES'], errors='coerce')


In [77]:
# Fix inconsistencies

# Remove any rows where END_DATE < START_DATE (invalid trip durations)
invalid_dates = df[df['END_DATE'] < df['START_DATE']].shape[0]
df = df[df['END_DATE'] >= df['START_DATE']]
print(f"Removed {invalid_dates} trips with inconsistent timestamps.")

# Remove negative or unrealistic miles (like 0 or > 500)
outliers = df[(df['MILES'] <= 0) | (df['MILES'] > 500)]
print(f"Found {len(outliers)} potential distance outliers.")
df = df[(df['MILES'] > 0) & (df['MILES'] <= 500)]



Removed 0 trips with inconsistent timestamps.
Found 0 potential distance outliers.


In [79]:
# Derive new useful columns

# Add trip duration in minutes
df['TRIP_DURATION_MIN'] = (df['END_DATE'] - df['START_DATE']).dt.total_seconds() / 60

# Extract month, day, and hour for demand analysis
df['MONTH'] = df['START_DATE'].dt.month_name()
df['DAY_OF_WEEK'] = df['START_DATE'].dt.day_name()
df['HOUR'] = df['START_DATE'].dt.hour


In [81]:
# Validation checks

assert df['MILES'].isna().sum() == 0, "❌ Miles column still has NaN values!"
assert (df['TRIP_DURATION_MIN'] >= 0).all(), "❌ Negative trip duration detected!"
assert df.duplicated().sum() == 0, "❌ Duplicate rows still exist!"
print("✅ Validation checks passed successfully.")

✅ Validation checks passed successfully.


In [83]:
# Save cleaned dataset

CLEAN_PATH.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(CLEAN_PATH, index=False)
print(f"✅ Cleaned dataset saved to: {CLEAN_PATH}")
print(f"Final dataset shape: {df.shape}")

✅ Cleaned dataset saved to: C:\Users\pooja\OneDrive\Desktop\PoojaPattu_E_DA_asses\UberDataset_Cleaned.csv
Final dataset shape: (420, 11)


In [85]:
#  Document data quality summary

print("\n📊 Data Quality Summary:")
print(f"Total Records: {len(df)}")
print(f"Columns: {list(df.columns)}")
print(f"Date Range: {df['START_DATE'].min()} to {df['START_DATE'].max()}")
print(f"Unique Trip Purposes: {df['PURPOSE'].nunique()}")
print("Sample Purposes:", df['PURPOSE'].unique()[:5])


📊 Data Quality Summary:
Total Records: 420
Columns: ['START_DATE', 'END_DATE', 'CATEGORY', 'START', 'STOP', 'MILES', 'PURPOSE', 'TRIP_DURATION_MIN', 'MONTH', 'DAY_OF_WEEK', 'HOUR']
Date Range: 2016-01-01 21:11:00 to 2016-12-12 20:48:00
Unique Trip Purposes: 7
Sample Purposes: ['Meal/Entertain' 'Unknown' 'Errand/Supplies' 'Meeting' 'Customer Visit']
